In [1]:
import pandas as pd

# 🔹 Path dyal fichier (kima 3andek)
input_path = "../data/Unified Dataset/unified_dataset_validated.csv"

# Ila bghiti tجرّب مباشرة b file li kayn 3andek f environment daba:
# input_path = "/mnt/data/unified_dataset_validated.csv"

# 🔹 Read CSV
df = pd.read_csv(input_path)

print("Rows before:", len(df))
print("Value counts of source (top):")
print(df["source"].value_counts(dropna=False).head(20))

# 🔹 Remove rows where source == "hypothyroid"
df_clean = df[df["source"].astype(str).str.lower() != "hypothyroid"].copy()

print("Rows after:", len(df_clean))
print("Removed rows:", len(df) - len(df_clean))

# 🔹 Save result (choose output)
output_path = "../data/Unified Dataset/unified_dataset_no_hypothyroid.csv"
df_clean.to_csv(output_path, index=False)

print("✅ Saved to:", output_path)

C:\Users\Click\AppData\Local\Temp\ipykernel_2288\2849639321.py:10: DtypeWarning: Columns (10,11,12,13,14,15,18,19,20,21,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path)


Rows before: 225568
Value counts of source (top):
source
cancer_risk    212691
thyroidDF        9172
hypothyroid      3705
Name: count, dtype: int64
Rows after: 221863
Removed rows: 3705
✅ Saved to: ../data/Unified Dataset/unified_dataset_no_hypothyroid.csv


In [2]:
df.shape

(225568, 37)

In [3]:
df_clean.shape[0]

221863